# Example

```python
import requests
url = 'https://...'
payload = {'key1': 'value1', 'key2': 'value2'}

# GET
r = requests.get(url)

# GET with params in URL
r = requests.get(url, params=payload)

# POST with form-encoded data
r = requests.post(url, data=payload)

# POST with JSON 
import json
r = requests.post(url, data=json.dumps(payload))

# Response, status etc
r.text
r.status_code
```

# Test Calculate API

In [48]:
import requests
import json
import numpy as np

def testResponse(r, statusCode=200):
    assert r.status_code == statusCode

In [49]:
HOST = 'http://david-pc:8000'
BASE_PATH = 'v1/calculator'
WITH_HOST = HOST + '/' + BASE_PATH

## Get all functions

In [50]:
url = WITH_HOST + "/functions"
r = requests.get(url)

testResponse(r)
d = json.loads(r.text)
assert len(list(d.keys())) > 0

for v in d.values():
    assert type(v) == list
    assert len(v) > 0
    
print(d)

{'TALIB': ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'BETA', 'CORREL', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR', 'AVGPRICE', 'MEDPRICE', 'TYPPRICE', 'WCLPRICE', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS', 'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH', '

## Get Functions Info

In [51]:
def testInfoSucces(url):
    r = requests.get(url)

    testResponse(r)

    d = json.loads(r.text)
    assert len(list(d.keys())) == 1
    assert type(list(d.values())[0]) == str
    print(list(d.values())[0])

### Test Success

In [52]:
url = WITH_HOST + "/functions/TALIB/AD/info"
testInfoSucces(url)

AD([input_arrays])

Chaikin A/D Line (Volume Indicators)

Inputs:
    prices: ['high', 'low', 'close', 'volume']
Outputs:
    real


In [53]:
url = WITH_HOST + "/functions/talib/ad/info"
testInfoSucces(url)

AD([input_arrays])

Chaikin A/D Line (Volume Indicators)

Inputs:
    prices: ['high', 'low', 'close', 'volume']
Outputs:
    real


### Test Not implemented function

In [54]:
url = WITH_HOST + "/functions/TALIB/NOT_IMPLEMENTED_FUNCTION/info"
r = requests.get(url)

testResponse(r, 400)

d = json.loads(r.text)
print(d['status'])
print(d['description'])

400 Bad Request
The "{'functionName': 'NOT_IMPLEMENTED_FUNCTION', 'group': 'TALIB'} info:Function NOT_IMPLEMENTED_FUNCTION of group TALIB is not implemented." parameter is invalid. invalid param(custom validation all), values:


## Calculate Talib from Abstract API

In [55]:
payload = {
    'data': {
        'open': np.random.random(100).tolist(),
        'close': np.random.random(100).tolist(),
        'high': np.random.random(100).tolist(), 
        'low': np.random.random(100).tolist(),
        'volume': np.random.random(100).tolist()
    },
    'args': [
        {'name': 'timeperiod', 'value': '25'}
    ]
        
}

In [56]:
def testCalculateSuccess(url):
    r = requests.post(url, data=json.dumps(payload))
    testResponse(r)
    d = json.loads(r.text)
    upper, middle, lower = d['result']
    assert len(upper) == len(middle) == len(lower) == 100

### Test success

In [57]:
url = WITH_HOST + "/functions/TALIB/BBANDS"
testCalculateSuccess(url)

In [58]:
url = WITH_HOST + "/functions/talib/bbands"
testCalculateSuccess(url)

### Test Fail

In [59]:
url = WITH_HOST + "/functions/TALIB/NOT_IMPLEMENTED_FUNCTION"
r = requests.post(url, data=json.dumps(payload))
testResponse(r, 500)
d = json.loads(r.text)
print(d['description'])

NOT_IMPLEMENTED_FUNCTION not supported by TA-LIB.


In [60]:
url = WITH_HOST + "/functions/NOT_IMPLEMENTED_GROUP/NOT_IMPLEMENTED_FUNCTION"
r = requests.post(url, data=json.dumps(payload))
testResponse(r, 500)
d = json.loads(r.text)
print(d['description'])

Group NOT_IMPLEMENTED_GROUP is not implemented by API.


In [61]:
r.status_code

500